Using SIFT

In [3]:
import cv2 as cv, numpy as np
from skimage import io

def sift_detector(new_image, image_template):
    """
    function that computes input image to template.
    It can return the number of SIFT matches between them
    """
    new_image = cv.cvtColor(new_image, cv.COLOR_BGR2GRAY)
    template = image_template
    
    # create SIFT detector
    sift = cv.SIFT_create()
    
    # obtain keypoints and descriptors
    keypoints1, descriptors1 = sift.detectAndCompute(new_image, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image_template, None)
    
    # define parameters for Flann Matcher
    FLANN_INDEX_KDTREE = 0
    index_parameters = dict(algorithm=FLANN_INDEX_KDTREE, trees=3)
    search_parameters = dict(checks=100)
    
    #create flann Matcher object
    flann = cv.FlannBasedMatcher(index_parameters, search_parameters)
    
    #obtain matches using k-nearest neighbour method, and result is:
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    
    # store good matches using Lowe's ratio test
    good_matches = []
    for m,n in matches:
        if m.distance < .7 * n.distance:
            good_matches.append(m)
    return len(good_matches)



In [4]:
cap = cv.VideoCapture(0)
image_template = cv.imread("C:/opencv/images/box_in_scene.png")

while True:
#     get webcam images
    ret, frame = cap.read()
    
    #get height and width of webcam frame
    height, width = frame.shape[:2]
    
    # define ROI box dimensions
    x1 = int(width/3)
    y1 = int(3*height/4)
    x2 = int(2*x1)
    y2 = int(y1/3)
    
    #draw rectangular window for our region of interest
    cv.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
    #then crop the window observation
    cropped = frame[y2:y1, x2:x1]
    
    #flip frame orientation horizontally
    frame = cv.flip(frame, 1)
    
    #get num of SIFT matches
    matches = sift_detector(cropped, image_template)
    
    # display status string showing the current no of matches
    cv.putText(frame, str(matches), (450,450), cv.FONT_HERSHEY_COMPLEX, 2, (0,255,0), 1)
    
    #threshold to indicate object detection, use 10 since SIFT detector returns little false positives
    threshold = 10
    
    # object is detected if matches > 10
    if matches > threshold:
        cv.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        cv.putText(frame, "Match found!", (450,450), cv.FONT_HERSHEY_COMPLEX, 2, (0,255,0), 1)
        
    cv.imshow("Object detector using SIFT", frame)
    if cv.waitKey(1) == 13: #enter key code is 13
        break
    
cap.release()
cv.destroyAllWindows()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


USING ORB

In [22]:
import cv2 as cv, numpy as np
# from skimage import io

def ORB_detector(new_image, image_template):
    """
    function that computes input image to template.
    It can return the number of ORB matches between them
    """
    image = cv.cvtColor(new_image, cv.COLOR_BGR2GRAY)
    
    # create SIFT detector
    orb = cv.ORB(1000, 1.2)
    
    
    
    # obtain keypoints and descriptors
    (keypoints1, descriptors1) = orb.detectAndCompute(new_image, None)
    (keypoints2, descriptors2) = orb.detectAndCompute(image_template, None)
    
    # define parameters..Matcher
    bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=2)
    
    #create flann Matcher object
    matches = bf.match(descriptors1, descriptors2)
    
    matches = sorted(matches, key=lambda val: val.distance)
    
    #obtain matches using k-nearest neighbour method, and result is:
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    
    return len(matches)



In [25]:
cap = cv.VideoCapture(0)
image_template = cv.imread("C:/opencv/images/box_in_scene.png")

while True:
    ret, frame = cap.read()
    height, width = frame.shape[:2]
    
    # define ROI box dimensions
    x1 = int(width/3)
    y1 = int(3*height/4)
    x2 = int(2*x1)
    y2 = int(y1/3)
    
    #draw rectangular window for our region of interest
    cv.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
    #then crop the window observation
    cropped = frame[y2:y1, x2:x1]
    
    #flip frame orientation horizontally
    frame = cv.flip(frame, 1)
    
    matches = ORB_detector(cropped, image_template)
    
    
    threshold = 350
    
    if matches > threshold:
        cv.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        cv.putText(frame, "Match found!", (450,450), cv.FONT_HERSHEY_COMPLEX, 2, (0,255,0), 1)
        
    cv.imshow("Object detector using ORB", frame)
    if cv.waitKey(1) == 13: #enter key code is 13
        break
    
cap.release()
cv.destroyAllWindows()

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-u4kjpz2z\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
